# 2030 Contract Case

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

import nygrid.nygrid as ng_grid
import nygrid.run_nygrid as ng_run
import nygrid.plots as ng_plots
import nygrid.postprocessing as ng_post

In [ ]:
# Set up directories
cwd = os.getcwd()
if 'examples' in cwd:
    parent_dir = os.path.dirname(cwd)
    data_dir = os.path.join(parent_dir, 'data')
else:
    data_dir = os.path.join(cwd, 'data')

grid_data_dir = os.path.join(data_dir, 'grid', '2030ContractCase')
if not os.path.exists(grid_data_dir):
    raise FileNotFoundError('Grid data directory not found.')
print('Grid data directory: {}'.format(grid_data_dir))

fig_dir = os.path.join(os.path.dirname(data_dir), 'figures')
print('Figure directory: {}'.format(fig_dir))

results_dir = os.path.join(os.path.dirname(data_dir), 'results')
print('Results directory: {}'.format(results_dir))

thermal_data_dir = os.path.join(data_dir, 'thermal')
if not os.path.exists(thermal_data_dir):
    raise FileNotFoundError('Thermal data directory not found.')

## Read grid data

### Read generator profiles

In [ ]:
start_date = datetime(2018, 1, 1, 0, 0, 0)
end_date = datetime(2019, 1, 1, 0, 0, 0)
timestamp_list = pd.date_range(start_date, end_date, freq='1D')
verbose = True

ext_cost_factor = 0.0
fo_cost_factor = 0.5

# Simulation time settings
valid_days = 7
lookahead_days = 1

valid_hours = 24 * valid_days
lookahead_hours = 24 * lookahead_days

options = {
    'UsePTDF': True,
    'solver': 'gurobi',
    'PenaltyForLoadShed': 10_000,
    'PenaltyForBranchMwViolation': 5_000,
    'PenaltyForInterfaceMWViolation': 5_000
}

solver_options = {
    'NodefileStart': 0.5,
    'NodefileDir': results_dir,
    'MIPGap': 1e-3
}

In [ ]:
# Read grid property file
grid_prop = ng_run.read_grid_prop(grid_data_dir,
                                  if_lims_prop_file='if_lims_prop_2030ContractCase.csv',
                                  esr_prop_file='esr_prop_2030ContractCase.csv',
                                  dcline_prop_file='dcline_prop_2030ContractCase.csv')

# Read load and generation profiles
grid_profile = ng_run.read_grid_profile(grid_data_dir, year=2030)

In [ ]:
# Decrease external generation cost by 50%
change_index = grid_prop["gen_prop"]["GEN_ZONE"].isin(
    ['PJM','IESO', 'NE']).to_numpy()

gencost1_profile_new = grid_profile['gencost1_profile'].copy()
gencost1_profile_new.loc[:, change_index] = \
    gencost1_profile_new.loc[:, change_index] * ext_cost_factor
grid_profile['gencost1_profile'] = gencost1_profile_new

In [ ]:
# Increase FO2, KER and BIT generation costs
change_index = grid_prop["gen_fuel"]["GEN_FUEL"].isin(
    [
        # "CC_NG", 
        # "CT_NG", 
        "CT_FO2", 
        "CT_KER", 
        # "ST_NG", 
        "ST_FO6", 
        # "ST_BIT"
        ]).to_numpy()

gencost0_profile_new = grid_profile['gencost0_profile'].copy()
gencost0_profile_new.loc[:, change_index] = gencost0_profile_new.loc[:, change_index] * fo_cost_factor
grid_profile['gencost0_profile'] = gencost0_profile_new

gencost1_profile_new = grid_profile['gencost1_profile'].copy()
gencost1_profile_new.loc[:, change_index] = gencost1_profile_new.loc[:, change_index] * fo_cost_factor
grid_profile['gencost1_profile'] = gencost1_profile_new

gencost_startup_profile_new = grid_profile['gencost_startup_profile'].copy()
gencost_startup_profile_new.loc[:, change_index] = gencost_startup_profile_new.loc[:, change_index] * fo_cost_factor
grid_profile['gencost_startup_profile'] = gencost_startup_profile_new

In [ ]:
gen_fuel = grid_prop['gen_fuel'].copy()
gen_fuel_dict = gen_fuel.set_index('GEN_NAME')['GEN_FUEL'].to_dict()

gencost0_profile_new = grid_profile['gencost0_profile'].copy()
gencost0_by_fuel = gencost0_profile_new.groupby(gen_fuel_dict, axis=1).mean()

gencost1_profile_new = grid_profile['gencost1_profile'].copy()
gencost1_by_fuel = gencost1_profile_new.groupby(gen_fuel_dict, axis=1).mean()

gencost_startup_profile_new = grid_profile['gencost_startup_profile'].copy()
gencost_startup_by_fuel = gencost_startup_profile_new.groupby(gen_fuel_dict, axis=1).mean()

fig, ax = plt.subplots()
gencost0_by_fuel.plot(ax=ax,
                      color=plt.cm.tab20.colors, 
                      title='Generation no-load cost by fuel type')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

fig, ax = plt.subplots()
gencost1_by_fuel.plot(ax=ax,
                      color=plt.cm.tab20.colors, 
                      title='Generation linear cost by fuel type')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

fig, ax = plt.subplots()
gencost_startup_by_fuel.plot(ax=ax,
                      color=plt.cm.tab20.colors, 
                      title='Generation startup cost by fuel type')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

## Multi-period OPF

### Jan. 1-8, 2018

In [ ]:
# Only run for one day
d = 0
start_datetime_w0 = timestamp_list[d]
end_datetime_w0 = start_datetime_w0 + timedelta(hours=valid_hours + lookahead_hours)
print(f'Start time: {start_datetime_w0}')
print(f'End time: {end_datetime_w0}')

In [ ]:
# Create NYGrid object
nygrid_sim_w0 = ng_grid.NYGrid(grid_prop=grid_prop,
                               start_datetime=start_datetime_w0,
                               end_datetime=end_datetime_w0,
                               verbose=verbose)

# Set load and generation time series data
nygrid_sim_w0.set_load_sch(grid_profile['load_profile'])
# nygrid_sim_0.set_gen_mw_sch(grid_profile['gen_profile'])
nygrid_sim_w0.set_gen_max_sch(grid_profile['genmax_profile'])
nygrid_sim_w0.set_gen_min_sch(grid_profile['genmin_profile'])
# nygrid_sim_0.set_gen_ramp_sch(grid_profile['genramp30_profile'])
nygrid_sim_w0.set_gen_cost_sch(grid_profile['gencost0_profile'],
                               grid_profile['gencost1_profile'])
nygrid_sim_w0.set_gen_cost_startup_sch(grid_profile['gencost_startup_profile'])
# nygrid_sim_0.set_gen_cost_shutdown_sch(grid_profile['gencost_shutdown_profile'])

if 'genmax_profile_vre' in grid_profile:
    nygrid_sim_w0.set_vre_max_sch(grid_profile['genmax_profile_vre'])
else:
    print('No VRE generation data.')

# Relax branch flow limits
nygrid_sim_w0.relax_external_branch_lim()

# Set generator initial condition
init_gen = np.zeros(nygrid_sim_w0.NG)
nygrid_sim_w0.set_gen_init_data(gen_init=init_gen)

# Set generator initial unit commitment condition
init_gen_commit = np.zeros(sum(grid_prop['gen_prop']['CMT_KEY'] == 1))
nygrid_sim_w0.set_gen_init_cmt_data(gen_init_cmt=init_gen_commit)

# Set options
nygrid_sim_w0.set_options(options)

# Solve DC OPF
nygrid_sim_w0.solve_dc_opf(solver_options=solver_options)

# Get results
results_w0 = nygrid_sim_w0.get_results_dc_opf()

In [ ]:
print("s_ramp_up", results_w0['s_ramp_up'].sum())
print("s_ramp_down", results_w0['s_ramp_down'].sum())
print("s_over_gen", results_w0['s_over_gen'].sum())
print("s_load_shed", results_w0['s_load_shed'].sum())
print("s_if_max", results_w0['s_if_max'].sum())
print("s_if_min", results_w0['s_if_min'].sum())
print("s_br_max", results_w0['s_br_max'].sum())
print("s_br_min", results_w0['s_br_min'].sum())
print("s_esr_pcrg", results_w0['s_esr_pcrg'].sum())
print("s_esr_pdis", results_w0['s_esr_pdis'].sum())
print("s_esr_soc_min", results_w0['s_esr_soc_min'].sum())
print("s_esr_soc_max", results_w0['s_esr_soc_max'].sum())
print("s_esr_soc_overt", results_w0['s_esr_soc_overt'].sum())
print("s_esr_soc_undert", results_w0['s_esr_soc_undert'].sum())
print("s_min_up_time", results_w0['s_min_up_time'].sum())
print("s_min_down_time", results_w0['s_min_down_time'].sum())

In [ ]:
# import pickle
# # Save simulation nygrid_results to pickle
# filename = f'nygrid_sim_2018_jan_1-8.pkl'
# with open(os.path.join(results_dir, filename), 'wb') as f:
#     pickle.dump(results_w0, f)

In [ ]:
# Only run for one day
d = d + valid_days
start_datetime_w1 = timestamp_list[d]
end_datetime_w1 = start_datetime_w1 + timedelta(hours=valid_hours + lookahead_hours)
print(f'Start time: {start_datetime_w1}')
print(f'End time: {end_datetime_w1}')

In [ ]:
# Create NYGrid object
nygrid_sim_w1 = ng_grid.NYGrid(grid_prop=grid_prop,
                            start_datetime=start_datetime_w1,
                            end_datetime=end_datetime_w1,
                            verbose=verbose)

# Set load and generation time series data
nygrid_sim_w1.set_load_sch(grid_profile['load_profile'])
# nygrid_sim_0.set_gen_mw_sch(grid_profile['gen_profile'])
nygrid_sim_w1.set_gen_max_sch(grid_profile['genmax_profile'])
nygrid_sim_w1.set_gen_min_sch(grid_profile['genmin_profile'])
# nygrid_sim_0.set_gen_ramp_sch(grid_profile['genramp30_profile'])
nygrid_sim_w1.set_gen_cost_sch(grid_profile['gencost0_profile'],
                            grid_profile['gencost1_profile'])
nygrid_sim_w1.set_gen_cost_startup_sch(grid_profile['gencost_startup_profile'])
# nygrid_sim_0.set_gen_cost_shutdown_sch(grid_profile['gencost_shutdown_profile'])

if 'genmax_profile_vre' in grid_profile:
    nygrid_sim_w1.set_vre_max_sch(grid_profile['genmax_profile_vre'])
else:
    print('No VRE generation data.')

# Relax branch flow limits
nygrid_sim_w1.relax_external_branch_lim()

# Set initial conditions for the next iteration
hour_before_start = start_datetime_w1 + timedelta(hours=-1)

# Set generator initial condition
init_gen = results_w0['PG'].loc[hour_before_start].to_numpy().squeeze()
nygrid_sim_w1.set_gen_init_data(gen_init=init_gen)

# Set generator initial unit commitment condition
init_gen_commit = results_w0['genCommit'].loc[hour_before_start].to_numpy().squeeze()
nygrid_sim_w1.set_gen_init_cmt_data(gen_init_cmt=init_gen_commit)

# Set ESR initial condition for the next iteration
init_soc = results_w0['esrSOC'].loc[hour_before_start].to_numpy().squeeze()
nygrid_sim_w1.set_esr_init_data(esr_init=init_soc)

# Calculate hours since last startup and shutdown
hour_since_last_startup = np.zeros(nygrid_sim_w0.NG_avail, dtype=int)
hour_since_last_shutdown = np.zeros(nygrid_sim_w0.NG_avail, dtype=int)

for ii in range(nygrid_sim_w0.NG_avail):
    startup_record = results_w0['genStartup'][:hour_before_start][ii]
    if startup_record.sum() > 0:
        last_startup_hour = startup_record[startup_record > 0].index[-1]
        hour_since_last_startup[ii] = int((hour_before_start - last_startup_hour).total_seconds()/ 3600)
    else:
        hour_since_last_startup[ii] = 999
    
    shutdown_record = results_w0['genShutdown'][:hour_before_start][ii]
    if shutdown_record.sum() > 0:
        last_shutdown_hour = shutdown_record[shutdown_record > 0].index[-1]
        hour_since_last_shutdown[ii] = int((hour_before_start - last_shutdown_hour).total_seconds()/3600)
    else:
        hour_since_last_shutdown[ii] = 999

nygrid_sim_w1.set_gen_last_startup_data(hour_since_last_startup)
nygrid_sim_w1.set_gen_last_shutdown_data(hour_since_last_shutdown)

# Set options
nygrid_sim_w1.set_options(options)

# Solve DC OPF
nygrid_sim_w1.solve_dc_opf(solver_options)

# Get results
results_w1 = nygrid_sim_w1.get_results_dc_opf()

In [ ]:
print("s_ramp_up", results_w1['s_ramp_up'].sum())
print("s_ramp_down", results_w1['s_ramp_down'].sum())
print("s_over_gen", results_w1['s_over_gen'].sum())
print("s_load_shed", results_w1['s_load_shed'].sum())
print("s_if_max", results_w1['s_if_max'].sum())
print("s_if_min", results_w1['s_if_min'].sum())
print("s_br_max", results_w1['s_br_max'].sum())
print("s_br_min", results_w1['s_br_min'].sum())
print("s_esr_pcrg", results_w1['s_esr_pcrg'].sum())
print("s_esr_pdis", results_w1['s_esr_pdis'].sum())
print("s_esr_soc_min", results_w1['s_esr_soc_min'].sum())
print("s_esr_soc_max", results_w1['s_esr_soc_max'].sum())
print("s_esr_soc_overt", results_w1['s_esr_soc_overt'].sum())
print("s_esr_soc_undert", results_w1['s_esr_soc_undert'].sum())
print("s_min_up_time", results_w1['s_min_up_time'].sum())
print("s_min_down_time", results_w1['s_min_down_time'].sum())

### Jul. 2-9, 2018

In [ ]:
# Only run for one day
d = 182
start_datetime_s0 = timestamp_list[d]
end_datetime_s0 = start_datetime_s0 + timedelta(hours=valid_hours + lookahead_hours)
print(f'Start time: {start_datetime_s0}')
print(f'End time: {end_datetime_s0}')

In [ ]:
# Create NYGrid object
nygrid_sim_s0 = ng_grid.NYGrid(grid_prop=grid_prop,
                            start_datetime=start_datetime_s0,
                            end_datetime=end_datetime_s0,
                            verbose=verbose)

# Set load and generation time series data
nygrid_sim_s0.set_load_sch(grid_profile['load_profile'])
# nygrid_sim_0.set_gen_mw_sch(grid_profile['gen_profile'])
nygrid_sim_s0.set_gen_max_sch(grid_profile['genmax_profile'])
nygrid_sim_s0.set_gen_min_sch(grid_profile['genmin_profile'])
# nygrid_sim_0.set_gen_ramp_sch(grid_profile['genramp30_profile'])
nygrid_sim_s0.set_gen_cost_sch(grid_profile['gencost0_profile'],
                            grid_profile['gencost1_profile'])
nygrid_sim_s0.set_gen_cost_startup_sch(grid_profile['gencost_startup_profile'])
# nygrid_sim_0.set_gen_cost_shutdown_sch(grid_profile['gencost_shutdown_profile'])

if 'genmax_profile_vre' in grid_profile:
    nygrid_sim_s0.set_vre_max_sch(grid_profile['genmax_profile_vre'])
else:
    print('No VRE generation data.')

# Relax branch flow limits
nygrid_sim_s0.relax_external_branch_lim()

# Set generator initial condition
init_gen = np.zeros(nygrid_sim_s0.NG)
nygrid_sim_s0.set_gen_init_data(gen_init=init_gen)

# Set generator initial unit commitment condition
init_gen_commit = np.zeros(nygrid_sim_s0.NG_avail)
nygrid_sim_s0.set_gen_init_cmt_data(gen_init_cmt=init_gen_commit)

# Set options
nygrid_sim_s0.set_options(options)

# Solve DC OPF
nygrid_sim_s0.solve_dc_opf(solver_options)

# Get results
results_s0 = nygrid_sim_s0.get_results_dc_opf()

In [ ]:
print("s_ramp_up", results_s0['s_ramp_up'].sum())
print("s_ramp_down", results_s0['s_ramp_down'].sum())
print("s_over_gen", results_s0['s_over_gen'].sum())
print("s_load_shed", results_s0['s_load_shed'].sum())
print("s_if_max", results_s0['s_if_max'].sum())
print("s_if_min", results_s0['s_if_min'].sum())
print("s_br_max", results_s0['s_br_max'].sum())
print("s_br_min", results_s0['s_br_min'].sum())
print("s_esr_pcrg", results_s0['s_esr_pcrg'].sum())
print("s_esr_pdis", results_s0['s_esr_pdis'].sum())
print("s_esr_soc_min", results_s0['s_esr_soc_min'].sum())
print("s_esr_soc_max", results_s0['s_esr_soc_max'].sum())
print("s_esr_soc_overt", results_s0['s_esr_soc_overt'].sum())
print("s_esr_soc_undert", results_s0['s_esr_soc_undert'].sum())
print("s_min_up_time", results_s0['s_min_up_time'].sum())
print("s_min_down_time", results_s0['s_min_down_time'].sum())

In [ ]:
# import pickle
# # Save simulation nygrid_results to pickle
# filename = f'nygrid_sim_2018_jul_2-9.pkl'
# with open(os.path.join(results_dir, filename), 'wb') as f:
#     pickle.dump(results_0, f)

In [ ]:
# Only run for one day
d = d + valid_days
start_datetime_s1= timestamp_list[d]
end_datetime_s1 = start_datetime_s1 + timedelta(hours=valid_hours + lookahead_hours)
print(f'Start time: {start_datetime_s1}')
print(f'End time: {end_datetime_s1}')

In [ ]:
# Create NYGrid object
nygrid_sim_s1 = ng_grid.NYGrid(grid_prop=grid_prop,
                            start_datetime=start_datetime_s1,
                            end_datetime=end_datetime_s1,
                            verbose=verbose)

# Set load and generation time series data
nygrid_sim_s1.set_load_sch(grid_profile['load_profile'])
# nygrid_sim_0.set_gen_mw_sch(grid_profile['gen_profile'])
nygrid_sim_s1.set_gen_max_sch(grid_profile['genmax_profile'])
nygrid_sim_s1.set_gen_min_sch(grid_profile['genmin_profile'])
# nygrid_sim_0.set_gen_ramp_sch(grid_profile['genramp30_profile'])
nygrid_sim_s1.set_gen_cost_sch(grid_profile['gencost0_profile'],
                            grid_profile['gencost1_profile'])
nygrid_sim_s1.set_gen_cost_startup_sch(grid_profile['gencost_startup_profile'])
# nygrid_sim_0.set_gen_cost_shutdown_sch(grid_profile['gencost_shutdown_profile'])

if 'genmax_profile_vre' in grid_profile:
    nygrid_sim_s1.set_vre_max_sch(grid_profile['genmax_profile_vre'])
else:
    print('No VRE generation data.')

# Relax branch flow limits
nygrid_sim_s1.relax_external_branch_lim()

# Set initial conditions for the next iteration
hour_before_start = start_datetime_s1 + timedelta(hours=-1)

# Set generator initial condition
init_gen = results_s0['PG'].loc[hour_before_start].to_numpy().squeeze()
nygrid_sim_s1.set_gen_init_data(gen_init=init_gen)

# Set generator initial unit commitment condition
init_gen_commit = results_s0['genCommit'].loc[hour_before_start].to_numpy().squeeze()
nygrid_sim_s1.set_gen_init_cmt_data(gen_init_cmt=init_gen_commit)

# Set ESR initial condition for the next iteration
init_soc = results_s0['esrSOC'].loc[hour_before_start].to_numpy().squeeze()
nygrid_sim_s1.set_esr_init_data(esr_init=init_soc)

# Calculate hours since last startup and shutdown
for ii in range(nygrid_sim_s0.NG_avail):
    startup_record = results_s0['genStartup'][:hour_before_start][ii]
    if startup_record.sum() > 0:
        last_startup_hour = startup_record[startup_record > 0].index[-1]
        hour_since_last_startup[ii] = int((hour_before_start - last_startup_hour).total_seconds()/ 3600)
    else:
        hour_since_last_startup[ii] = 999
    
    shutdown_record = results_s0['genShutdown'][:hour_before_start][ii]
    if shutdown_record.sum() > 0:
        last_shutdown_hour = shutdown_record[shutdown_record > 0].index[-1]
        hour_since_last_shutdown[ii] = int((hour_before_start - last_shutdown_hour).total_seconds()/3600)
    else:
        hour_since_last_shutdown[ii] = 999

nygrid_sim_s1.set_gen_last_startup_data(hour_since_last_startup)
nygrid_sim_s1.set_gen_last_shutdown_data(hour_since_last_shutdown)

# Set options
nygrid_sim_s1.set_options(options)

# Solve DC OPF
nygrid_sim_s1.solve_dc_opf(solver_options)

# Get results
results_s1 = nygrid_sim_s1.get_results_dc_opf()

In [ ]:
print("s_ramp_up", results_s1['s_ramp_up'].sum())
print("s_ramp_down", results_s1['s_ramp_down'].sum())
print("s_over_gen", results_s1['s_over_gen'].sum())
print("s_load_shed", results_s1['s_load_shed'].sum())
print("s_if_max", results_s1['s_if_max'].sum())
print("s_if_min", results_s1['s_if_min'].sum())
print("s_br_max", results_s1['s_br_max'].sum())
print("s_br_min", results_s1['s_br_min'].sum())
print("s_esr_pcrg", results_s1['s_esr_pcrg'].sum())
print("s_esr_pdis", results_s1['s_esr_pdis'].sum())
print("s_esr_soc_min", results_s1['s_esr_soc_min'].sum())
print("s_esr_soc_max", results_s1['s_esr_soc_max'].sum())
print("s_esr_soc_overt", results_s1['s_esr_soc_overt'].sum())
print("s_esr_soc_undert", results_s1['s_esr_soc_undert'].sum())
print("s_min_up_time", results_s1['s_min_up_time'].sum())
print("s_min_down_time", results_s1['s_min_down_time'].sum())

## Process the results

### Read CEMS data

In [ ]:
gen_combiner = pd.read_excel(os.path.join(thermal_data_dir, '2018_nyca_thermal.xlsx'),
                             sheet_name='matched_with_id_filtered')
gen_combiner = gen_combiner.rename(columns={
    '   Station        Unit': 'NYISO_Name',
})

# Map unit type GT, JE to CT
gen_combiner['Unit Type'] = gen_combiner['Unit Type'].replace(
    {'GT': 'CT', 'JE': 'CT'})
gen_combiner['Fuel Type Secondary'] = gen_combiner['Fuel Type Secondary'].astype(
    str).replace({'nan': ''})

gen_combiner['GEN_NAME'] = gen_combiner['NYISO_Name'] + gen_combiner['ID']
gen_combiner

In [ ]:
cems_data_dir = os.path.join(thermal_data_dir, 'cems_2018')
df_list = list()
for mo in range(1, 13):
    df = pd.read_csv(os.path.join(cems_data_dir, f'2018ny{str(mo).zfill(2)}.csv'),
                     low_memory=False)
    df_list.append(df)

# Concatenate all the dataframes
cems_df = pd.concat(df_list, axis=0, ignore_index=True)

# Rename columns
cems_df = cems_df.rename(columns={
    'ORISPL_CODE': 'Plant_ID',
    'UNITID': 'Unit_ID',
})

# Add time columns from OP_DATE and OP_HOUR
cems_df['Time'] = pd.to_datetime(
    cems_df['OP_DATE']) + pd.to_timedelta(cems_df['OP_HOUR'], unit='h')
# cems_df['Month'] = cems_df['Time'].dt.month

# Drop columns that are not needed
cems_df = cems_df.drop(columns=['STATE', 'FAC_ID', 'UNIT_ID', 'OP_DATE', 'OP_HOUR',
                                'SO2_MASS_MEASURE_FLG', 'SO2_RATE_MEASURE_FLG',
                                'NOX_MASS_MEASURE_FLG', 'NOX_RATE_MEASURE_FLG',
                                'CO2_MASS_MEASURE_FLG', 'CO2_RATE_MEASURE_FLG',
                                'SLOAD (1000lb/hr)'])

In [ ]:
cems_gload = dict()
cems_heat_input = dict()

for ii in range(gen_combiner.shape[0]):

    gen_info = gen_combiner.iloc[ii]

    unit_df = cems_df[(cems_df['Plant_ID'] == gen_info['CAMD_Plant_ID'])
                        & (cems_df['Unit_ID'] == gen_info['CAMD_Unit_ID'])]
    unit_df = unit_df.set_index('Time').sort_index()
    
    cems_gload[gen_info['GEN_NAME']] = unit_df['GLOAD (MW)']
    # cems_heat_input[gen_info['GEN_NAME']] = unit_df['HEAT_INPUT (mmBtu)']

# Create a dataframe from the dictionary
cems_gload_df = pd.DataFrame(cems_gload)
# cems_heat_input_df = pd.DataFrame(cems_heat_input)

# Subset the data to the time range of the simulation
cems_gload_df_w0 = cems_gload_df.loc[start_datetime_w0:end_datetime_w0]
cems_gload_df_w1 = cems_gload_df.loc[start_datetime_w1:end_datetime_w1]
cems_gload_df_s0 = cems_gload_df.loc[start_datetime_s0:end_datetime_s0]
# cems_gload_df_s1 = cems_gload_df.loc[start_datetime_s1:end_datetime_s1]

In [ ]:
gen_fuel = gen_combiner[['GEN_NAME','Unit Type', 'Fuel Type Primary']].copy()
gen_fuel['GEN_FUEL'] = gen_fuel['Unit Type'] + '_' + gen_fuel['Fuel Type Primary']
gen_fuel_dict = gen_fuel.set_index('GEN_NAME')['GEN_FUEL'].to_dict()

# GlOAD grouped by unit type and fuel type
cems_gload_summary_w0 = cems_gload_df_w0.groupby(gen_fuel_dict, axis=1).sum()
cems_gload_summary_w1 = cems_gload_df_w1.groupby(gen_fuel_dict, axis=1).sum()
cems_gload_summary_s0 = cems_gload_df_s0.groupby(gen_fuel_dict, axis=1).sum()
# cems_gload_summary_s1 = cems_gload_df_s1.groupby(gen_fuel_dict, axis=1).sum()

### Power dispatch

In [ ]:
# Power dispatch of generators by fuel type
# Including ESR, DC line, and VRE
pg_by_fuel_dict_w0 = ng_post.get_pg_by_fuel(results_w0, nygrid_sim_w0,
                                              valid_hours=nygrid_sim_w0.NT)

# Drop NJ generators
cols_drop_cc_ng = [col for col in pg_by_fuel_dict_w0['CC_NG'].columns if '50006' in col]
pg_by_fuel_dict_w0['CC_NG'] = pg_by_fuel_dict_w0['CC_NG'].drop(columns=cols_drop_cc_ng)

cols_drop_ct_ng = [col for col in pg_by_fuel_dict_w0['CT_NG'].columns if '56964' in col]
pg_by_fuel_dict_w0['CT_NG'] = pg_by_fuel_dict_w0['CT_NG'].drop(columns=cols_drop_ct_ng)

# 2018 Baseline
pg_by_fuel_sum_w0 = dict()
for fuel_type, pg_by_fuel in pg_by_fuel_dict_w0.items():
    pg_by_fuel_sum_w0[fuel_type] = pg_by_fuel.sum(axis=1)

# Convert to DataFrame
pg_by_fuel_sum_w0 = pd.DataFrame(pg_by_fuel_sum_w0,
                                   index=pg_by_fuel_dict_w0['CC_NG'].index)


In [ ]:
# Power dispatch of generators by fuel type
# Including ESR, DC line, and VRE
pg_by_fuel_dict_w1 = ng_post.get_pg_by_fuel(results_w1, nygrid_sim_w1,
                                              valid_hours=nygrid_sim_w1.NT)

# Drop NJ generators
cols_drop_cc_ng = [col for col in pg_by_fuel_dict_w1['CC_NG'].columns if '50006' in col]
pg_by_fuel_dict_w1['CC_NG'] = pg_by_fuel_dict_w1['CC_NG'].drop(columns=cols_drop_cc_ng)

cols_drop_ct_ng = [col for col in pg_by_fuel_dict_w1['CT_NG'].columns if '56964' in col]
pg_by_fuel_dict_w1['CT_NG'] = pg_by_fuel_dict_w1['CT_NG'].drop(columns=cols_drop_ct_ng)

# 2018 Baseline
pg_by_fuel_sum_w1 = dict()
for fuel_type, pg_by_fuel in pg_by_fuel_dict_w1.items():
    pg_by_fuel_sum_w1[fuel_type] = pg_by_fuel.sum(axis=1)

# Convert to DataFrame
pg_by_fuel_sum_w1 = pd.DataFrame(pg_by_fuel_sum_w1,
                                   index=pg_by_fuel_dict_w1['CC_NG'].index)


In [ ]:
# Power dispatch of generators by fuel type
# Including ESR, DC line, and VRE
pg_by_fuel_dict_s0 = ng_post.get_pg_by_fuel(results_s0, nygrid_sim_s0,
                                           valid_hours=nygrid_sim_s0.NT)

# Drop NJ generators
cols_drop_cc_ng = [col for col in pg_by_fuel_dict_s0['CC_NG'].columns if '50006' in col]
pg_by_fuel_dict_s0['CC_NG'] = pg_by_fuel_dict_s0['CC_NG'].drop(columns=cols_drop_cc_ng)

cols_drop_ct_ng = [col for col in pg_by_fuel_dict_s0['CT_NG'].columns if '56964' in col]
pg_by_fuel_dict_s0['CT_NG'] = pg_by_fuel_dict_s0['CT_NG'].drop(columns=cols_drop_ct_ng)

# 2018 Baseline
pg_by_fuel_sum_s0 = dict()
for fuel_type, pg_by_fuel in pg_by_fuel_dict_s0.items():
    pg_by_fuel_sum_s0[fuel_type] = pg_by_fuel.sum(axis=1)

# Convert to DataFrame
pg_by_fuel_sum_s0 = pd.DataFrame(pg_by_fuel_sum_s0,
                                   index=pg_by_fuel_dict_s0['CC_NG'].index)

In [ ]:
# Power dispatch of generators by fuel type
# Including ESR, DC line, and VRE
pg_by_fuel_dict_s1 = ng_post.get_pg_by_fuel(results_s1, nygrid_sim_s1,
                                           valid_hours=nygrid_sim_s0.NT)

# Drop NJ generators
cols_drop_cc_ng = [col for col in pg_by_fuel_dict_s1['CC_NG'].columns if '50006' in col]
pg_by_fuel_dict_s1['CC_NG'] = pg_by_fuel_dict_s1['CC_NG'].drop(columns=cols_drop_cc_ng)

cols_drop_ct_ng = [col for col in pg_by_fuel_dict_s1['CT_NG'].columns if '56964' in col]
pg_by_fuel_dict_s1['CT_NG'] = pg_by_fuel_dict_s1['CT_NG'].drop(columns=cols_drop_ct_ng)

# 2018 Baseline
pg_by_fuel_sum_s1 = dict()
for fuel_type, pg_by_fuel in pg_by_fuel_dict_s1.items():
    pg_by_fuel_sum_s1[fuel_type] = pg_by_fuel.sum(axis=1)

# Convert to DataFrame
pg_by_fuel_sum_s1 = pd.DataFrame(pg_by_fuel_sum_s1,
                                   index=pg_by_fuel_dict_s1['CC_NG'].index)

In [ ]:
# External bus load profile
bus_prop = nygrid_sim_w0.grid_prop['bus_prop']
bus_zone_dict = bus_prop.set_index('BUS_I')['BUS_ZONE'].to_dict()

load_profile_by_zone_2018 = grid_profile['load_profile'].groupby(bus_zone_dict, axis=1).sum()

load_profile_external = load_profile_by_zone_2018[['NE','PJM','IESO']]
load_profile_external_sum = load_profile_external.sum(axis=1)

load_profile_external_sum_w0 = load_profile_external_sum.loc[start_datetime_w0:end_datetime_w0]
load_profile_external_sum_w1 = load_profile_external_sum.loc[start_datetime_w1:end_datetime_w1]
load_profile_external_sum_s0 = load_profile_external_sum.loc[start_datetime_s0:end_datetime_s0]
# load_profile_external_sum_s1 = load_profile_external_sum.loc[start_datetime_s1:end_datetime_s1]

net_import_w0 = pg_by_fuel_sum_w0['Import_Import'] - load_profile_external_sum_w0
net_import_w1 = pg_by_fuel_sum_w1['Import_Import'] - load_profile_external_sum_w1
net_import_s0 = pg_by_fuel_sum_s0['Import_Import'] - load_profile_external_sum_s0
# net_import_s1 = pg_by_fuel_sum_s1['Import_Import'] - load_profile_external_sum_s1

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(12, 8), sharex=True,
                        layout='constrained')

for i, fuel_type in enumerate(cems_gload_summary_w0.columns):
    if fuel_type == 'ST_BIT':
        continue
    ax = axs.flatten()[i]
    cems_gload_summary_w0[fuel_type].plot(ax=ax, alpha=0.5, color='red',
                                          label='CEMS d1', marker='.')
    cems_gload_summary_w1[fuel_type].plot(ax=ax, alpha=0.5, color='tab:red',
                                          label='CEMS d2', marker='.')
    pg_by_fuel_sum_w0[fuel_type].plot(ax=ax, alpha=0.5, color='blue',
                                      label='NYGrid d1', marker='.')
    pg_by_fuel_sum_w1[fuel_type].plot(ax=ax, alpha=0.5, color='tab:blue',
                                      label='NYGrid d2', marker='.')

    ax.set_title(fuel_type)
    ax.legend(loc='upper right')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)

ax = axs.flatten()[-1]
net_import_w0.plot(ax=ax, alpha=0.5, color='blue',
                   label='NYGrid', marker='.')
net_import_w1.plot(ax=ax, alpha=0.5, color='tab:blue',
                   label='NYGrid', marker='.')
ax.set_title('Net Import')
ax.legend(loc='upper right')

fig.supxlabel('Time')
fig.supylabel('Power (MW)')
fig.suptitle('Total generation by fuel type (Winter)')
plt.show()

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(12, 8), sharex=True,
                        layout='constrained')

for i, fuel_type in enumerate(cems_gload_summary_s0.columns):
    if fuel_type == 'ST_BIT':
        continue
    ax = axs.flatten()[i]
    cems_gload_summary_s0[fuel_type].plot(ax=ax, alpha=0.5, color='red',
                                          label='CEMS d1', marker='.')
    # cems_gload_summary_s1[fuel_type].plot(ax=ax, alpha=0.5, color='tab:red',
    #                                       label='CEMS d2', marker='.')
    pg_by_fuel_sum_s0[fuel_type].plot(ax=ax, alpha=0.5, color='blue',
                                      label='NYGrid d1', marker='.')
    # pg_by_fuel_sum_s1[fuel_type].plot(ax=ax, alpha=0.5, color='tab:blue',
    #                                   label='NYGrid d2', marker='.')

    ax.set_title(fuel_type)
    ax.legend(loc='upper right')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)

ax = axs.flatten()[-1]
net_import_s0.plot(ax=ax, alpha=0.5, color='blue',
                   label='NYGrid', marker='.')
# net_import_s1.plot(ax=ax, alpha=0.5, color='tab:blue',
#                    label='NYGrid', marker='.')
ax.set_title('Net Import')
ax.legend(loc='upper right')

fig.supxlabel('Time')
fig.supylabel('Power (MW)')
fig.suptitle('Total generation by fuel type (Summer)')
plt.show()

In [ ]:
pg_by_fuel_sum_w0.plot(subplots=True, layout=(8, 2), figsize=(12, 8), sharex=True)

In [ ]:
pg_by_fuel_sum_s0.plot(subplots=True, layout=(8, 2), figsize=(12, 8), sharex=True)

### Unit commitment

In [ ]:
n_thermal = nygrid_sim_w0.NG - nygrid_sim_w0.NESR - nygrid_sim_w0.NDCL*2
gen_pg = results_w0['PG'].iloc[:, :n_thermal]
gen_commit = results_w0['genCommit'].iloc[:, :n_thermal]
gen_startup = results_w0['genStartup'].iloc[:, :n_thermal]
gen_shutdown = results_w0['genShutdown'].iloc[:, :n_thermal]


In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(8, 6), sharex=True, layout='constrained')
axs[0].plot(gen_commit, label='gen_commit')
axs[1].plot(gen_startup, label='gen_startup')
axs[2].plot(gen_shutdown, label='gen_shutdown')
axs[3].plot(gen_pg, label='gen_pg')
axs[0].set_ylabel('gen_commit')
axs[1].set_ylabel('gen_startup')
axs[2].set_ylabel('gen_shutdown')
axs[3].set_ylabel('gen_pg')
for ax in axs[:-1]:
    ax.set_ylim(-0.1, 1.1)
    ax.grid(linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
ii = 100

fig, axs = plt.subplots(4, 1, figsize=(8, 6), sharex=True, layout='constrained')
axs[0].plot(gen_commit[ii], marker='.', label='gen_commit')
axs[1].plot(gen_startup[ii], marker='.', label='gen_startup')
axs[2].plot(gen_shutdown[ii], marker='.', label='gen_shutdown')
axs[3].plot(gen_pg[ii], marker='.', label='gen_pg')
axs[0].set_ylabel('gen_commit')
axs[1].set_ylabel('gen_startup')
axs[2].set_ylabel('gen_shutdown')
axs[3].set_ylabel('gen_pg')
for ax in axs[:-1]:
    ax.set_ylim(-0.1, 1.1)
    ax.grid(linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
grid_prop['gen_prop'][['GEN_NAME', 'UNIT_TYPE', 'FUEL_TYPE', 'PMAX', 'MIN_UP_TIME', 'MIN_DOWN_TIME']].iloc[ii, :]

### Costs

In [ ]:
print(f'Total cost: {results_w0["total_cost"].sum():.2f}')
print(f'Total penalty: {results_w0["total_penalty"].sum():.2f}')
print(f'Total cost and penalty: {results_w0["total_cost_penalty"].sum():.2f}')
print(f'Penalty percentage: {results_w0["total_penalty"].sum()/results_w0["total_cost"].sum()*100:.2f}%')

print(f'Gen cost: {results_w0["gen_cost"].sum():.2f}')
print(f'Gen no load cost: {results_w0["gen_cost_noload"].sum():.2f}')
print(f'Gen startup cost: {results_w0["gen_cost_startup"].sum():.2f}')
print(f'Gen shutdown cost: {results_w0["gen_cost_shutdown"].sum():.2f}')
print(f'ESR cost: {results_w0["esr_cost"].sum():.2f}')

print(f'Over generation penalty: {results_w0["over_gen_penalty"].sum():.2f}')
print(f'Load shed penalty: {results_w0["load_shed_penalty"].sum():.2f}')
print(f'Ramp up penalty: {results_w0["ramp_up_penalty"].sum():.2f}')
print(f'Ramp down penalty: {results_w0["ramp_down_penalty"].sum():.2f}')
print(f'Interface max penalty: {results_w0["if_max_penalty"].sum():.2f}')
print(f'Interface min penalty: {results_w0["if_min_penalty"].sum():.2f}')
print(f'Branch max penalty: {results_w0["br_max_penalty"].sum():.2f}')
print(f'Branch min penalty: {results_w0["br_min_penalty"].sum():.2f}')

print(f'ESR Charging power penalty: {results_w0["esr_pcrg_penalty"].sum():.2f}')
print(f'ESR Discharging power penalty: {results_w0["esr_pdis_penalty"].sum():.2f}')
print(f'ESR SOC max penalty: {results_w0["esr_soc_max_penalty"].sum():.2f}')
print(f'ESR SOC min penalty: {results_w0["esr_soc_min_penalty"].sum():.2f}')
print(f'ESR SOC over target penalty: {results_w0["esr_soc_overt_penalty"].sum():.2f}')
print(f'ESR SOC under target penalty: {results_w0["esr_soc_undert_penalty"].sum():.2f}')


### ESR

In [ ]:
# ii = 7
# crg_max = grid_prop['esr_prop']['ESR_CRG_MAX'].iloc[ii]
# dis_max = grid_prop['esr_prop']['ESR_DIS_MAX'].iloc[ii]
# crg_eff = grid_prop['esr_prop']['ESR_CRG_EFF'].iloc[ii]
# dis_eff = grid_prop['esr_prop']['ESR_DIS_EFF'].iloc[ii]
# soc_init = grid_prop['esr_prop']['ESR_SOC_INI'].iloc[ii]
# soc_max = grid_prop['esr_prop']['ESR_SOC_MAX'].iloc[ii]

# print(f'crg_max: {crg_max}, dis_max: {dis_max}, crg_eff: {crg_eff}, dis_eff: {dis_eff}, soc_init: {soc_init}, soc_max: {soc_max}')


In [ ]:
# fig, axs = plt.subplots(3, 1, figsize=(10, 6),
#                        sharex=True, layout='constrained')

# results_0['esrPCrg'].iloc[:, ii].plot(ax=axs[0], label='Day 1 w/o IC', alpha=0.5)
# results_0['esrPDis'].iloc[:, ii].plot(ax=axs[1], label='Day 1 w/o IC', alpha=0.5)
# results_0['esrSOC'].iloc[:, ii].plot(ax=axs[2], label='Day 1 w/o IC', alpha=0.5)

# # results_1['esrPCrg'].iloc[:, ii].plot(ax=axs[0], label='Day 2 w/ IC', alpha=0.5)
# # results_1['esrPDis'].iloc[:, ii].plot(ax=axs[1], label='Day 2 w/ IC', alpha=0.5)
# # results_1['esrSOC'].iloc[:, ii].plot(ax=axs[2], label='Day 2 w/ IC', alpha=0.5)

# titles = ['Charge Power', 'Discharge Power', 'State of Charge']
# for i, ax in enumerate(axs):
#     ax.set_title(titles[i])